# Benchmark of sorted lists & sets implementations 

First, you may need to install the following packages

In [ ]:
!pip install "sortedcontainers>=2.2.2"
!pip install "blist>=1.3.6"
!pip install "tqdm>=4.39.0"
!pip install "Pympler>=0.8"
!pip install "py-cpuinfo>=6.0.0"
!pip install "pyroaring>=0.2.9"

If you have problems with the installation of pyroaring on macOS, run `brew install gcc@10`, `export CC=gcc-10 CXX=g++-10`, and then `pip install pyroaring`

## ⚙️ Set up the experiments

Here we set the `number` of times an operation is executed, the data `sizes` that must be tested, a function `gen_list_data` to generate data for the sorted list structures, and a function `gen_set_data` to generate data (without duplicates) for the set structures. Note that pyroaring requires `gen_set_data` to output 32-bit unsigned integers.

In [ ]:
import random
import numpy as np

number = 50000
sizes = [10 ** x for x in range(2, 8)]
gen_list_data = lambda s: np.sort(np.random.normal(0, 2 ** 40, s).astype(int)).tolist()
gen_set_data = lambda s: sorted(random.sample(range(min(2 ** 32, s * 1000)) , s))

Define the **list** structures and the operations to benchmark

In [ ]:
from collections import defaultdict
from pympler.asizeof import asizeof

import blist
import pygm
import pickle
import sortedcontainers

list_structures = {
    'sortedcontainers.SortedList' : {
        'ctor' : lambda data: sortedcontainers.SortedList(data),
        'mem' : lambda o : asizeof(o)
    },
    'pygm.SortedList' : {
        'ctor' : lambda data: pygm.SortedList(data, 'q'),
        'mem' : lambda o: o.stats()['data size'] + o.stats()['index size']
    },
    'blist.sortedlist' : {
        'ctor' : lambda data: blist.sortedlist(data),
        'mem' : lambda o : len(pickle.dumps(o, -1)) + o._blist.__sizeof__()
    }
}

list_experiments = defaultdict(dict)
for struct in list_structures:
    list_experiments['__init__'][struct] = {
        'func' : lambda _, data, ctor=list_structures[struct]['ctor']: ctor(data),
        'args' : lambda data: data,
        'limit' : 1
    }
    
    list_experiments['__contains__ ⟺ x in list'][struct] = {
        'func' : '__contains__',
        'args' : lambda data: random.choice(data)
    }
    
    list_experiments['__getitem__ ⟺ list[i]'][struct] = {
        'func' : '__getitem__',
        'args' : lambda data: random.randint(0, len(data) - 1)
    }
    
    list_experiments['index'][struct] = {
        'func' : 'index',
        'args' : lambda data: data[random.randint(0, len(data) - 1)]
    }
    
    list_experiments['count'][struct] = {
        'func' : 'count',
        'args' : lambda data: random.choice(data)
    }
    
    list_experiments['bisect_left'][struct] = {
        'func' : 'bisect_left',
        'args' : lambda data: random.randint(data[0], data[len(data) - 1])
    }

Define the **set** structures and the operations to benchmark

In [ ]:
import pyroaring

from functools import partial


def get_set_data(data, factor):
    n = max(int(len(data) * factor), 10) // 2
    i = random.randint(0, len(data) - n)
    return list(range(i, i + n)) + list(random.sample(data, n))


set_structures = {
    'sortedcontainers.SortedSet' : {
        'ctor' : lambda data: sortedcontainers.SortedSet(data),
        'mem' : lambda o : asizeof(o)
    },
    'pygm.SortedSet' : {
        'ctor' : lambda data: pygm.SortedSet(data, 'I'),
        'mem' : lambda o: o.stats()['data size'] + o.stats()['index size']
    },
    'pyroaring.BitMap' : {
        'ctor' : lambda data: pyroaring.BitMap(data),
        'mem' : lambda o: o.__sizeof__()
    },
}

override_index = {'pyroaring.BitMap': lambda self, x: self.rank(x)}
override_bisect = {'pyroaring.BitMap': lambda self, x: self.rank(x)}
override_args = {
    'sortedcontainers.SortedSet': lambda f, data: sortedcontainers.SortedSet(get_set_data(data, f)),
    'pygm.SortedSet': lambda f, data: pygm.SortedSet(get_set_data(data, f), 'I'),
    'pyroaring.BitMap': lambda f, data: pyroaring.BitMap(get_set_data(data, f)),
}

set_experiments = defaultdict(dict)
for struct in set_structures:
    set_experiments['__init__'][struct] = {
        'func' : lambda _, data, ctor=set_structures[struct]['ctor']: ctor(data),
        'args' : lambda data: data,
        'limit' : 1
    }
    
    set_experiments['__contains__ ⟺ x in set'][struct] = {
        'func' : '__contains__',
        'args' : lambda data: random.choice(data)
    }
    
    set_experiments['__getitem__ ⟺ set[i]'][struct] = {
        'func' : '__getitem__',
        'args' : lambda data: random.randint(0, len(data) - 1)
    }
    
    set_experiments['index'][struct] = {
        'func' : override_index.get(struct, 'index'),
        'args' : lambda data: data[random.randint(0, len(data) - 1)]
    }
    
    set_experiments['bisect_left'][struct] = {
        'func' : override_bisect.get(struct, 'bisect_left'),
        'args' : lambda data: random.randint(data[0], data[len(data) - 1])
    }
    
    for op in ['union', 'difference', 'intersection', 'symmetric_difference']:        
        set_experiments[op + ' (small)'][struct] = {
            'func' : op,
            'args' : partial(override_args[struct], 1 / 100),
            'limit' : 1
        }

        set_experiments[op + ' (medium)'][struct] = {
            'func' : op,
            'args' : partial(override_args[struct], 1 / 10),
            'limit' : 1
        }

        set_experiments[op + ' (large)'][struct] = {
            'func' : op,
            'args' : partial(override_args[struct], 9 / 10),
            'limit' : 1
        }

## ⏳ Measure the performance

In [ ]:
import gc

from time import time
from tqdm.notebook import tqdm
from functools import partial


def run_exp(structures, experiments, data_generator):
    mem_results = defaultdict(list)
    results = defaultdict(lambda: defaultdict(list))
    pbar = tqdm(total=len(sizes) * len(structures) * len(experiments))

    for size in sizes:
        data = data_generator(size)
        for struct in structures:
            gc.collect()
            obj = structures[struct]['ctor'](data)
            mem = structures[struct]['mem'](obj)
            mem_results[struct].append(mem)
            
            for exp in experiments:
                info = experiments[exp][struct]
                limit = info.get('limit', number)
                args = info['args']
                func = info['func']
                func = getattr(obj, func) if isinstance(func, str) else partial(func, obj)

                sec = 0
                batch_size = min(1000, limit)
                for _ in range(0, limit, batch_size):
                    batch = [args(data) for _ in range(batch_size)]
                    gc.disable()
                    t0 = time()
                    for x in batch:
                        func(x)
                    t1 = time()
                    sec += t1 - t0
                    gc.enable()
                
                results[exp][struct].append(sec / ((limit // batch_size) * batch_size) )
                pbar.update(1)

    pbar.close()
    return results, mem_results


list_results, mem_list_results = run_exp(list_structures, list_experiments, gen_list_data)
set_results, mem_set_results = run_exp(set_structures, set_experiments, gen_set_data)

## 📈 Plot performance results

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
from math import ceil


def platform_info():
    import platform as p
    from cpuinfo import get_cpu_info
    return '%s  ~^~  %s  ~^~  %s %s (%s)' % (
        get_cpu_info()['brand_raw'],
        p.platform(),
        p.python_implementation(), 
        p.python_version(),
        p.python_compiler(),)


def plot(results, structures, title):
    colors = ['C%d' % i for i in range(10)]
    markers = ['o', '^', '>', 's', '<', 'p', 'P', 'D']
    ncols = min(3, ceil(len(results) ** 0.5))
    nrows = ceil(len(results) / ncols)
    figsize = (5.3 * ncols, 4.5 * nrows)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, squeeze=False,
                            sharex=False, sharey=False, constrained_layout=True)
    fig.suptitle(title, fontsize=20, weight='medium')
    fig.text(0.5, -0.5 / figsize[1], platform_info(), fontsize=10, family='monospace',
             ha='center')

    for (exp_i, (exp, ax)) in enumerate(zip(results, axs.flat)):
        for i, struct in enumerate(structures):
            ax.plot(sizes, results[exp][struct],
                    label=struct, color=colors[i], marker=markers[i])

        if exp_i >= ncols * (nrows - 1):
            ax.set_xlabel('Data size')
        if exp_i % ncols == 0:
            ax.set_ylabel('Time')
        ax.legend(loc='upper left')
        ax.set_title(exp)
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.yaxis.set_major_formatter(ticker.EngFormatter(unit='s'))
    
    #plt.savefig(title + '.svg', bbox_inches='tight')
    plt.show()


plot(list_results, list_structures, 'Performance of sorted lists')
plot(set_results, set_structures, 'Performance of sorted sets')

## 📊 Plot the memory usage

In [ ]:
def plot_mem(results, structures, title):
    fig, ax = plt.subplots(constrained_layout=True)
    bars = len(sizes) * len(structures)
    x = np.arange(len(sizes))
    group_spacing = 1.2
    width = len(sizes) / (bars * group_spacing)
    fig.suptitle(title, fontsize=18, weight='medium')

    for i, struct in enumerate(structures):
        offset = x + width * i - 0.5 * width * (len(structures) - 1)
        ax.bar(offset, results[struct], label=struct, width=width)

    fmt = ticker.ScalarFormatter(useMathText=True)
    ax.set_xticks(x)
    ax.set_xticklabels(['$%s$' % fmt.format_data(x) for x in sizes])
    ax.set_yscale('log')
    ax.set_xlabel('Data size')
    ax.set_ylabel('Memory')
    ax.yaxis.set_major_formatter(ticker.EngFormatter(unit='B'))
    ax.legend()
    
    #plt.savefig(title + '.svg', bbox_inches='tight')
    plt.show()


plot_mem(mem_list_results, list_structures, 'Memory usage of sorted lists')
plot_mem(mem_set_results, set_structures, 'Memory usage of sorted sets')

## Explore PGM segments

In [ ]:
import matplotlib.pyplot as plt

def plot_segments(pgm_idx, level=0):
    
    seg0 = pgm_idx.segment(level, 0)
    plt.title(f"level={level} eps={int(seg0['epsilon'])}")
    
    segments_count = pgm_idx.num_segments(level)
    for seg_num in range(0, segments_count):
        seg = pgm_idx.segment(level, seg_num)

        x_start = seg['key']

        if seg_num == segments_count - 1:
            x_stop = pgm_idx[-1]
        else:
            x_stop = pgm_idx.segment(level, seg_num+1)['key']

        x_vals = np.array([ x_start, x_stop ])
        y_vals = seg['intercept'] + seg['slope'] * (x_vals - seg['key'])

        eps = seg['epsilon']
        plt.gca().fill_between(x_vals, y_vals - eps, y_vals + eps, alpha=0.5, label=f'segment {seg_num}')

    if level > 0:
        prev_level = level - 1
        prev_level_segments_count = pgm_idx.num_segments(prev_level)
        intercepts = []
        keys = []
        for seg_num in range(0, prev_level_segments_count):
            seg = pgm_idx.segment(prev_level, seg_num)
            intercepts.append(seg['intercept'])
            keys.append(seg['key'])

        plt.scatter(keys, range(len(keys)), marker='X', label=f'level {prev_level} keys')

In [ ]:
pygm_idx = pygm.SortedList(gen_list_data(1000), epsilon=16)

In [ ]:
plot_segments(pygm_idx, level=0)

plt.scatter(pygm_idx, range(len(pygm_idx)), s=1, alpha=0.1, color='blue')

plt.gcf().set_size_inches((15,5))

In [ ]:
plot_segments(pygm_idx, level=1)
plt.gcf().set_size_inches((15,5))